In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import datetime
import re
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CREATE DATA

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

final_status = train.final_status
projest_id = train.project_id
backers_count = train.backers_count

ltr = len(train)
train.drop(['final_status', 'backers_count'], axis = 1, inplace = True)

data = pd.concat([train, test], axis = 0)
data.index = range(len(data))

In [3]:
int_disable_communication = []
for x in data.disable_communication.tolist():
    if x == False:
        int_disable_communication += [0]
    else:
        int_disable_communication += [1]
data['disable_communication'] = int_disable_communication

In [4]:
data['deadline-created_at'] = data.deadline - data.created_at
data['launched_at-created_at'] = data.deadline - data.created_at
data['state_changed_at-created_at'] = data.deadline - data.created_at
data['state_changed_at-deadline'] = data.state_changed_at - data.deadline
data['deadline-launched_at'] = data.deadline - data.launched_at
data['state_changed_at-launched_at'] = data.state_changed_at - data.launched_at

In [5]:
data['len_name'] = [len(str(x)) for x in data.name.tolist()]
data['len_desc'] = [len(str(x)) for x in data.desc.tolist()]
data['len_keywords'] = [len(str(x)) for x in data.keywords.tolist()]
data['numb_keywords'] = [len(str(x).split('-')) for x in data.keywords.tolist()]

In [6]:
len_cov = []
for x in data.desc.tolist():
    tokens = re.findall('\"', str(x))
    len_cov += [len(tokens)]
data['len_cov'] = len_cov
data['bad_znak'] =  data['len_cov'] / data['len_desc']

In [7]:
normal_goal = []
for x, y in zip(data.currency.tolist(), data.goal.tolist()):
    if x == 'USD':
        normal_goal += [y]
    if x == 'GBP':
        normal_goal += [1.5 * y]
    if x == 'EUR':
        normal_goal += [1.2 * y]
    if x == 'CAD':
        normal_goal += [0.85 * y]
    if x == 'AUD':
        normal_goal += [0.85 * y]
    if x == 'SEK':
        normal_goal += [0.14 * y]
    if x == 'NZD':
        normal_goal += [0.70 * y]
    if x == 'DKK':
        normal_goal += [0.17 * y]
    if x == 'NOK':
        normal_goal += [0.15 * y]
    if x == 'CHF':
        normal_goal += [y]
    if x == 'MXN':
        normal_goal += [0.07 * y]
    if x == 'SGD':
        normal_goal += [0.73 * y]
    if x == 'HKD':
        normal_goal += [0.13 * y]

data['normal_goal'] = normal_goal
data['deadline-created_at_normal_goal'] = data.loc[:, 'deadline-created_at'] / data.normal_goal
data['launched_at-created_at_normal_goal'] = data.loc[:, 'launched_at-created_at'] / data.normal_goal
data['state_changed_at-created_at_normal_goal'] = data.loc[:, 'state_changed_at-created_at'] / data.normal_goal
data['state_changed_at-deadline_normal_goal'] = data.loc[:, 'state_changed_at-deadline'] / data.normal_goal
data['deadline-launched_at_normal_goal'] = data.loc[:, 'deadline-launched_at'] / data.normal_goal
data['state_changed_at-launched_at_normal_goal'] = data.loc[:, 'state_changed_at-launched_at'] / data.normal_goal

In [8]:
time_feat = ['deadline', 'created_at', 'launched_at',  'state_changed_at']
for time in time_feat:
    weekday = []
    hour = []
    day = []
    for x in data.loc[:, time].tolist():
        weekday += [datetime.datetime.fromtimestamp(x).weekday()]
        hour += [datetime.datetime.fromtimestamp(x).hour]
        day += [datetime.datetime.fromtimestamp(x).day]
    data[time + '_' + 'weekday'] = weekday
    data[time + '_' + 'hour'] = hour
    data[time + '_' + 'day'] = day
    
for time in time_feat:
    print(time + '_' + 'hour_weekday')
    data[time + '_' + 'hour_weekday'] = data[time + '_' + 'hour'].astype(str) + '_' + data[time + '_' + 'weekday'].astype(str)
    data[time + '_' + 'hour_country'] = data[time + '_' + 'hour'].astype(str) + '_' + data['country'].astype(str)
    data[time + '_' + 'weekday_country'] = data[time + '_' + 'weekday'].astype(str) + '_' + data['country'].astype(str)
    data[time + '_' + 'day_country'] = data[time + '_' + 'day'].astype(str) + '_' + data['country'].astype(str)

deadline_hour_weekday
created_at_hour_weekday
launched_at_hour_weekday
state_changed_at_hour_weekday


In [9]:
canceled = []
for x in data.name.tolist():
    if len(re.findall('Canceled', str(x))) > 0:
        canceled += [1]
    else:
        canceled += [0]
data['canceled'] = canceled

In [10]:
for x in ['deadline_hour_weekday',
'created_at_hour_weekday', 'launched_at_hour_weekday', 'state_changed_at_hour_weekday']:
    for y in ['country', 'currency']:
        data[x + y] = (data[x] + data[y]).astype('category').cat.codes

for x in ['deadline_hour_weekday',
'created_at_hour_weekday', 'launched_at_hour_weekday', 'state_changed_at_hour_weekday']:
    for y in ['country']:
        for z in ['currency']:
            data[x + y] = (data[x] + data[y] + data[z]).astype('category').cat.codes
            
for x in ['country', 'currency', 'deadline_hour_weekday',
'created_at_hour_weekday', 'launched_at_hour_weekday', 'state_changed_at_hour_weekday']:
    data[x] = data[x].astype('category').cat.codes

In [11]:
for x in ['deadline_hour_country', 'deadline_weekday_country', 'deadline_day_country', 'created_at_hour_country',
'created_at_weekday_country', 'created_at_day_country', 'launched_at_hour_country', 'launched_at_weekday_country',
'launched_at_day_country', 'state_changed_at_hour_country', 'state_changed_at_weekday_country', 'state_changed_at_day_country']:
    data[x] = data[x].astype('category').cat.codes

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

name = data.name.fillna('None').tolist()
tfidf = TfidfVectorizer(max_features = 500, stop_words = 'english', ngram_range = (1, 2))
name_vect = tfidf.fit_transform(name)

desc = data.desc.fillna('None').tolist()
tfidf = TfidfVectorizer(max_features = 2000, stop_words = 'english', ngram_range = (1, 4))
desc_vect = tfidf.fit_transform(desc)

keywords = data.keywords.tolist()
tfidf = TfidfVectorizer(max_features = 1000, stop_words = 'english', ngram_range = (1, 3))
keywords_vect = tfidf.fit_transform(keywords)

In [13]:
from scipy.sparse import hstack

sp_data = hstack([keywords_vect, name_vect, desc_vect]).tocsr()
del tfidf, keywords_vect, name_vect, desc_vect

LEARN MODEL

In [14]:
name = data.name.fillna('None').tolist()
tfidf = TfidfVectorizer(max_features = 2000, ngram_range = (1, 6), analyzer = 'char')
name_char = tfidf.fit_transform(name)

print(1)

desc = data.desc.fillna('None').tolist()
tfidf = TfidfVectorizer(max_features = 6000, ngram_range = (1, 6), analyzer = 'char')
desc_char = tfidf.fit_transform(desc)
print(1)
keywords = data.keywords.tolist()
tfidf = TfidfVectorizer(max_features = 3000, ngram_range = (1, 6), analyzer = 'char')
keywords_char = tfidf.fit_transform(keywords)

1
1


In [15]:
sp_data = hstack([sp_data, name_char, desc_char, keywords_char]).tocsr()
del tfidf, name_char, desc_char, keywords_char

In [16]:
pred = pd.DataFrame()
pred['svc'] = [-1] * (len(test))
clf = LinearSVC()
clf.fit(sp_data[:ltr], final_status)
pred['svc'] = clf.predict(sp_data[ltr:])

In [17]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C = 2)
clf.fit(sp_data[:ltr], final_status)
pred['logreg'] = clf.predict_proba(sp_data[ltr:])[:, -1]

In [18]:
data = np.array(data.drop(['project_id', 'name', 'desc', 'keywords'], axis = 1))
sp_data1 = hstack([sp_data, data]).tocsr()

In [19]:
del data, sp_data

In [20]:
dic_par = {'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error',
               'max_depth':5, 'subsample': 0.7, 'colsample_bytree': 0.8, 'min_child_weight': 5}
dtest = xgb.DMatrix(sp_data1[ltr:, 14500:])
dtrain = xgb.DMatrix(sp_data1[:ltr, 14500:], label=final_status)

bst = xgb.train(dic_par, dtrain, 222)
pred['xgb1'] = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [21]:
del dtrain, dtest, bst

In [22]:
dic_par = {'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error',
               'max_depth':8, 'subsample': 0.7, 'colsample_bytree': 0.7, 'min_child_weight': 5}
dtest = xgb.DMatrix(sp_data1[ltr:])
dtrain = xgb.DMatrix(sp_data1[:ltr], label=final_status)

bst = xgb.train(dic_par, dtrain, 416)
pred['xgb2'] = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [23]:
del dtrain, dtest, bst

In [24]:
dic_par = {'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error',
               'max_depth':7, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 6}
dtest = xgb.DMatrix(sp_data1[ltr:])
dtrain = xgb.DMatrix(sp_data1[:ltr], label=final_status)

bst = xgb.train(dic_par, dtrain, 228)
pred['xgb3'] = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [25]:
del dtrain, dtest, bst

In [26]:
import lightgbm as lgb

In [27]:
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'num_leaves': 80,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 2
}
lgb_train = lgb.Dataset(sp_data1[:ltr], np.array(final_status))
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=148)
pred['lgb1'] = gbm.predict(sp_data1[ltr:])

In [30]:
del lgb_train

In [31]:
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'num_leaves': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
}
lgb_train = lgb.Dataset(sp_data1[:ltr, 14500:], np.array(final_status))
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=124)
pred['lgb2'] = gbm.predict(sp_data1[ltr:, 14500:])

In [34]:
del lgb_train

READ PREDICT FROM ANOTHER FILE

In [35]:
pred_new = pd.read_csv('predict.csv', header = None)
pred['xgb_old'] = pred_new[1][ltr:].tolist()

ADD BAGGING(SPLIT BY TIME INTERVAL, COEF FROM LINEAR REGRESSION WITH NOISE)

In [36]:
final_pred = pd.Series([-100] * len(test))
col = ['logreg', 'xgb1', 'xgb2', 'xgb3', 'lgb1', 'lgb2', 'xgb_old']
new_col = [x + '1' for x in col]
pred[new_col] = pred[col].round()

In [37]:
date = pd.to_datetime(test.created_at,unit='s')

In [38]:
y2017 = date[date.dt.year == 2017].index
coef = [-0.027163179271743373, 0.50829964473673006, 0.21379022476789045, 0.21797444121259385, 
        0.10735714061372345, 0.17883463215622081, -0.11532700181862125, 0.0054078524855090682, 
        0.019044340523711512, 0.020904671380734853, 0.016799278763598158, -0.035555538613519899, 
        .070046648507590167, 0.015530628742049274, 0.041652250420829068]
final_pred[y2017] = (np.array(pred.loc[y2017, :]) * coef).sum(axis = 1).round()

In [39]:
y2010 = date[date.dt.year.isin([2010,2011,2012,2013,2014])].index

coef = [-0.027163179271743373, 0.50829964473673006, 0.21379022476789045, 0.21797444121259385, 0.10735714061372345, 
        0.17883463215622081, -0.11532700181862125, 0.0054078524855090682, 0.019044340523711512, 0.020904671380734853, 
        0.016799278763598158, -0.035555538613519899, 0.070046648507590167, 0.015530628742049274, 0.041652250420829068]
final_pred[y2010] = (np.array(pred.loc[y2010, :]) * coef).sum(axis = 1).round()

In [40]:
coef_list2015 = [[[-0.41383995384225819, -0.16307572346026833, -0.10896588031522875, 0.35471079322753063, 0.22390883115227922, -0.32863241243857799, 0.097549160081059405, 1.1372073765804023, 0.87153515112747171, -0.080863813424291636, 0.28487078798716486, -0.55400069424988796, -0.17659008257259162, 0.35160714276697574, -0.21936988430087989], -0.08629609882731698], [[-0.39286094196229099, 0.34510484198718872, 0.89787931598668014, -1.486627118144938, 1.245325850060452, 1.1053038953569081, -1.4454198657086255, 0.55721922803318291, 0.27085791705579459, 0.064604935954636344, 0.13771723388179469, -0.11899338973184959, -0.064443006982082052, 0.14364967167706721, -0.27374821149664608], 0.0036556793890552552], [[0.083346325658424161, -0.12831268413013974, 1.1959007032760487, -0.5233849336825912, 0.39900940712026656, 0.42745138413344769, -2.1143414710142512, 1.4761439331118933, 0.083346325658423981, -0.10015748315347639, -0.47897883986230105, 0.40147546444362403, 0.31978463221249886, 0.46676606100165352, -0.43921808570249288], 0.10734424730150077], [[-0.025017998270211974, 0.8149649690076205, 0.22134553385093006, -0.27829229874696421, 0.2163670138984588, 1.4214632555094264, 0.020124861567298286, -0.66832723045793496, -0.11756296346591055, 0.024269691750705785, 0.20962183305716242, -0.39990573074287339, -0.17063736111252792, 0.052916305919370354, 0.27456099230977582], -0.087124834756392822], [[-0.17857188656760381, -0.50018596299048845, 1.4635581312988146, -0.0504486329542157, 1.4202366056430444, 0.13200100751257016, -1.3792525358813492, -0.0043358756294259448, 0.61417616538769393, -0.091765531130930111, -0.1849941363492198, -0.28635895012223983, 0.08344268469493421, 0.029755702251762495, -0.12095137688527768], 0.059323755834263248], [[0.23266570483925128, 0.59183162350733021, 0.85228046871709484, -0.18994063134104572, -0.13074213291852804, -0.57422222869450446, -0.55415140450869049, 1.4479664887719843, -0.50511446356786971, -0.3152090047679722, -0.12621744624435693, 0.053571357532884112, 0.57767217434333518, 0.26510233281407025, -0.37662132192200815], 0.013015069835713933], [[0.079112999667759101, 0.53610646520732519, -1.155397987642315, 0.2565622200003016, 0.14382146435348217, 1.3073772692753969, 0.87854749768335161, -0.42129780803337913, -0.24246377511520789, -0.055366480326769274, -0.14171743379467075, 0.26797543629685394, 0.19325962796902985, 0.062853835080842801, -0.31020409520730441], -0.001021638893230814], [[-0.14140528990693546, -0.4157381903396174, 0.32643815251828778, 0.47829470762966453, 0.14540838047690763, -0.070953783338113974, 0.07259713275983698, 1.0273916737378574, 0.2105072342428041, -0.15848873991231688, 0.23483160420629368, -0.37459121068207463, 0.03737705965952931, 0.092195302291443681, -0.085222845716571793], 0.0078031063428684044], [[0.10550662841276728, 0.72612646807507764, 0.88080140234336735, -0.058116892446551854, -0.30759765699777342, 0.85370792127323414, -0.73696074165507164, 0.1901825008658854, -0.18077110440369148, -0.1083241428815005, 0.10296198995045747, 0.13142052324342346, -0.1441361965581997, 0.15228816196374362, -0.16334161610384135], -0.042526473797645292], [[0.033740498382682026, 0.080843506890849542, -0.083625278536957698, 0.24728436962909617, -0.55848674776303997, 0.83791202376532703, 0.048375547816753614, 1.1468314736516987, -0.069594498991358028, -0.055720452596246917, 0.037641457795474897, 0.043932420232981995, -0.049062634778156131, -0.012202354713624297, -0.21266052896534438], 0.0022399725905200008], [[-0.17409663626947336, 0.33843178141729097, -0.53125937805643997, 0.79958304675601433, 0.04106395228403914, 0.034285833381269976, 0.70360379309129073, 0.45037001128304066, -0.009205923702995028, -0.031796753471828643, 0.033143113104870336, -0.20158329715391332, 0.32201721875719203, -0.26813488809618247, -0.08392575310327377], -0.013782307499392965], [[-0.30701584284152922, 0.5942492683928583, 0.23239298742646675, 0.51924826923963019, -0.018419637046819481, 0.45063965311071841, 0.71939432972730277, -0.99474793277483242, 0.43454889983497075, -0.01691951305694267, -0.12875288169134289, -0.2424468901124931, -0.051384055344316615, -0.026109141686985748, 0.33732870822515598], -0.022383224751678088], [[0.15177273114974479, -0.026347320130969451, 0.25714275716769786, -0.60081133386779295, 1.5031631570715014, 0.43161693644893617, -0.29902155663521324, 0.098163961202380037, -0.15912121243201116, -0.19138392116446523, 0.13537943701624849, -0.1437814082267192, -0.0049560636108855194, 0.13559807851976091, 0.049246074378967775], 0.022938205951003654], [[0.083373645090151383, 0.44047294051683972, 0.74861778201841345, -0.20226722175100323, 0.87339331088680272, -0.27091310474858565, -0.6773592374808135, 0.28950885701085288, -0.11428941375895169, 0.064335136098343737, 0.085030349125940985, -0.10794608697245556, 0.056561839146722304, 0.24180383330031219, -0.11578746024682307], -0.046245635922416151], [[-0.050371576006364556, 0.6726724372394467, 0.25161075195261418, 0.67810570052226105, 0.59750443557138855, -0.4095965095982248, 0.087341165718058555, -0.01399886708101386, -0.1129046724188246, 0.10705832882972, -0.08347928741234778, 0.036823106059101052, -0.046602563791332902, -0.060134854954618455, 0.053886915926962181], -0.070925883612583063], [[0.077757796330699722, 0.29439865068850385, 0.43827025281740206, 0.35369973922058889, -0.078012553798705031, 0.075846743133362399, -0.45084175214799821, 0.77945829571817871, -0.031010811043684208, 0.0021613621474618205, 0.0078662533977770233, 0.10998001384693995, -0.14270865670379468, 0.061212808939159924, -0.17481214510521037], -0.0082442323680364527], [[-0.0064521578344324771, 0.41694436929706225, 0.20744985991777531, 0.35062836067520409, 0.29385875391622807, 0.16496024305653861, 0.064932604791711881, 0.25991103584696063, -0.1497593824171288, -0.18249832759070406, -0.14947770489950465, -0.10636999044390935, 0.1030929747886534, -0.0052910789341637787, 0.0053631808788744129], -0.050704163539114], [[0.22618710579684795, 0.31056171954317158, 0.21148323865181329, 0.56238236792633811, -0.38802270579054898, 0.50123862939552399, 0.013060952495983269, 0.42142316833453486, -0.14627618413647422, -0.00074587493405625649, -0.080276015934133693, -0.020160317076800549, -0.065571331961821161, 0.11343891541036422, -0.12186886136797415], -0.067370582777799337], [[-0.021166784463613177, 0.37550565405307484, 0.40151444164484273, 0.55020638162551849, -0.40732832938350261, 0.52349114862659396, 0.092195903521491318, 0.00076687090595081589, 0.056342743605521772, 0.072530193047288166, -0.025441056107622151, 0.11105505003182653, -0.15479516729304266, -0.039872341320651872, 0.018317039855248352], -0.068029487047780002], [[-0.038502206012433142, 0.1846914167264177, -0.012346366389744123, -0.32675224793022695, 0.32883359264341405, 0.80578704244427946, 0.20223654563194438, 0.2867340017678921, 0.050752578137505111, 0.073907401772084463, 0.049807673019682697, -0.055565730740980895, -0.19496891199125929, -0.028915831048797663, 0.062628046019031713], -0.048029081326567191], [[0.10091406807450043, 0.29974702524870739, 0.4129055100414003, 0.081063474592990709, 0.73313546834260235, -0.050258705754476329, -0.076959817628007077, 0.15423294085164146, -0.063825304014272444, -0.016442470293993372, -0.058599320299935864, -0.05472523946420732, 0.026006936119348345, -0.025964162544909519, -0.055132724117975374], -0.057617515387948792], [[-0.13412571009324611, 0.28867598295950708, 0.83654410646535038, 0.28744844076126608, 0.096612475021383093, 0.22575683170007652, -0.38651929812922864, 0.22053437132158993, 0.091632146031291467, -0.092885990626032311, -0.085645502553226185, 0.064797226467597363, 0.071972380240118095, 0.0043700370634170427, -0.013152856788972489], -0.053802826760923406], [[0.11128812962073018, 0.090281010573965148, 0.4713430340572165, 0.49463928492923137, -0.85945069178762168, 1.1062059214649451, -0.11803519286306985, 0.27940357132598365, -0.010427237053803481, 0.06616255841965657, 0.047939009754045747, 0.13430250169613434, -0.26490673303777479, -0.095745368665667463, -0.058540872952473344], -0.035069013726363951], [[0.022176366776143132, 0.26783831291371463, 0.28277588311753432, 0.47846058106773559, 0.16097025556464878, 0.2571532474206788, 0.012421942353549095, 0.070547552636659627, 0.0038074992477577507, 0.060850205792590858, 0.011059711916478789, -0.17663880153962472, -0.08402572264726911, -0.048057976864883084, 0.10959690375543049], -0.073200989594718324], [[-0.090304449224284669, 0.49505311698570725, 0.9053338838928614, 0.40652720285926353, -0.11490412096443998, -0.098850876691273903, -0.39369186142375756, 0.34103526425569303, 0.038250938244487998, -0.029405985065134366, -0.18598476424640928, 0.08084270069570032, 0.090540519485523802, 0.057132674845881665, 0.052513088783564199], -0.07959706965928609], [[0.034032949283043942, 0.43840900310101066, 0.42569503775973472, 0.028574249007546057, 0.37555142622684001, 0.35767653562159568, -0.11049481867598984, 0.2268268093364002, -0.10829224779425944, -0.056374962049798139, -0.056207087776516756, -0.13788653698481018, 0.11076960903459684, -0.0010955468465459473, -0.011783861552982333], -0.056547139659597401], [[-0.083736262982500836, 0.51790857341129104, 0.22985557359544206, 0.036319451048168069, 0.08837876334638084, 0.70331749885554329, 0.13589645438844405, 0.20851642148244737, 0.022701999775491616, 0.083971680331921317, -0.079447240778194567, -0.099434288371511095, -0.14378732684723694, -0.070311448520760128, 0.036857677991006033], -0.10836956431151584], [[-0.068557504813032605, 0.26378895686980636, -0.26641784069181468, 0.040062426713872612, 0.56711754869939868, 0.42277416390708517, 0.2490748737220656, 0.30443647171716692, 0.062113553145254408, 0.040751176294925917, 0.056732236572030637, -0.074881428530962302, 0.003889022629293859, -0.053635216449648804, -0.056706126189836753], -0.056701181353114083], [[-0.17546570273489157, 0.43384915148572484, 0.16508675957300636, 0.44127927336104517, -0.88372309184143083, 0.98728170988045361, 0.28839230234366586, 0.27573818316821119, 0.2000919673725336, 0.03301671253297378, 0.14169158656984532, -0.0041617381686023802, -0.27982885612923369, -0.10830269104552115, -0.05674728609894264], -0.060562733003362679], [[-0.07666996966260306, 0.47714699207326589, 0.22605680907628603, 0.31502624020937509, -0.36475049392442677, 0.60351163741844072, -0.019577217727722446, 0.19943273776652326, 0.043846737957801002, -0.017958288969666725, 0.13689204825568435, 0.061463101783768737, -0.19770044948964921, 0.10110510955148963, -0.012943943165161009], -0.05128653661657806], [[-0.16132807588827527, 0.41774138045343401, -0.015151873430680387, 0.25997315921009484, 0.5763079399248735, -0.1113946094014111, 0.20336140106543807, 0.21908978843202909, 0.14287374729900668, -0.068921910922539964, -0.15961689075722965, -0.020865388903675952, 0.064943390319233324, -0.020705354823424116, 0.10982466710622138], -0.066536479411353544], [[0.033420578920728417, 0.36261397183334343, 0.11522342338398239, 0.26811771127358552, 0.45928653716345469, 0.088297155469125865, 0.046621023088578961, 0.16860717733653874, -0.10814273405689259, 0.09233421397035077, 0.061115125945500309, -0.10779632497544339, -0.011971386582525445, -0.060966834403295866, 0.013015925716524157], -0.041185756159896558], [[-0.2001934238267373, 0.35756821822539775, 0.65629248341856783, 0.7222437647466009, -0.45124717573004536, 0.16166894454599889, -0.55679306764127956, 0.74961179323168303, 0.11106026591460433, -0.044603850777884424, -0.069731305019352152, 0.016434566356183844, -0.14644456070896128, 0.10611304915772002, 0.037700570463224614], -0.071258519696936196], [[-0.10295951257957371, 0.26497789321265153, 0.17735696503865289, 0.25004078842064581, -0.1647187228055734, 0.65231499049870001, 0.43516256847086809, 0.11431377802962625, 0.10800730125931264, -0.14992026834471867, -0.12102503699504252, 0.12357349660658856, -0.082886271233495457, -0.067460966622815949, 0.036317888288777633], -0.070039765613889027], [[-0.096907389037331454, 0.17050439167661569, 0.64205061346879988, 0.76313628034326009, -0.0044909688331003783, 0.30761665709503117, -0.31100357813475199, 0.025406843425650694, 0.14796448023372932, -0.083493968100144422, -0.17118020356075014, -0.024403338970001798, -0.092876865018402421, 0.071073147526174274, 0.047936643137305279], -0.038680520288416842], [[0.079948716058980601, 0.31351124735614011, 0.55810534252532806, 0.61721371456929985, -0.20591359843991136, 0.14379263217675714, -0.092754694036302099, 0.45850832554198534, -0.089644213069306686, -0.1481613738985047, -0.0027943169767884468, -0.06712279432969534, -0.035892366242658402, 0.07226570483180475, -0.10956944991592577], -0.06771292192303735], [[0.14208770949351235, 0.32634980833629473, 0.0057023557206580394, 0.34005409286447319, 0.023877414588673748, 0.13129096485728847, 0.51437288924768865, 0.23756857474314735, 0.0077691359060347506, 0.036860099004501906, -0.033258458545080499, -0.032523937334426434, 0.0010050016670497541, -0.077843564598096826, -0.10308283687025624], -0.071094499113193876], [[-0.077394610941101172, 0.30918759781497629, 0.16011474994892291, 0.15240543266999998, 0.30830256344156948, 0.61035118576443181, 0.096442516757869634, 0.14390346875654592, 0.033232887524917154, -0.16539139459578883, 0.0058115506692817398, -0.16757131562514824, 0.042536117455280131, 0.039348880899364724, 0.030092896671985392], -0.060883113904824093], [[-0.0070828350649748077, 0.40781712459741987, 0.034236102462881002, -0.01338658192344374, 0.22754362109380527, 0.38596225272624901, 0.52043963194730458, 0.24760818062480192, -0.076569855880416871, -0.11286547598304614, 0.086397646579394582, -0.015818525068393652, -0.13414010910793983, -0.055122509422700028, 0.060905993687274673], -0.071882279302774355], [[-0.02783460471464733, 0.32386775286017178, 0.40398860140344511, 0.24814403493961668, -0.0025605358775599857, 0.70320912952820713, -0.073975073500936711, -0.12049477833890768, 0.012364886108188731, -0.066584009851180681, 0.093800535507852445, -0.041253359718894045, -0.11733492720617113, 0.071131323927756285, 0.030340345313332573], -0.023072640349124429], [[0.054224530017478262, 0.35199437661707283, -0.085541348942181281, -0.12135721626161355, 0.49833798692878006, 0.55726615408398217, 0.28427948322998875, 0.41481215479310918, -0.0099423104735685275, 0.10074679708786802, -0.20725673275882073, 0.023772809400098571, 0.012800216072735904, -0.079181836826978824, -0.15337319851858133], -0.082871891843055989], [[-0.0042761698649839332, 0.43608710290235025, 0.18529779382451289, 0.084334868886577957, 0.30706194694592254, 0.68899336942951139, 0.10435512625136395, 0.0044062011365280651, -0.10907245389055792, -0.11987942604225243, 0.06016292047339275, -0.075457948154415255, -0.087176235287321269, 0.17738944525485981, 0.027081393019452407], -0.064576609537433494], [[-0.12606536049257239, 0.62386988399598142, -0.023261447248895589, -0.17343459301746444, -0.053332308429939107, 0.4354331749164882, 0.16103332381176294, 0.5337953514815662, -0.0063621913100341126, 0.00037921639833465859, 0.13656400106184058, 0.12478707533979005, -0.1434102084799333, 0.16504635366330389, -0.12793340951786364], -0.038997739327800818], [[0.023625106997053526, 0.21465053115274352, -0.10239683891827478, 0.55091010709219912, -0.19591912739509182, 0.58174125012288447, 0.14048587981746832, 0.35939100493920428, 0.0036477217904137549, 0.062895547806887275, -0.025414936857275305, -0.16024746218096914, -0.05692126463352748, 0.095222291902597428, -0.0013976777143698271], -0.021205154288806594], [[-0.1415733161297775, 0.19282706324156584, -0.15317008495995021, 0.41760498989039097, 0.22934896395910748, 0.099855835777710789, 0.86143296168583372, 0.079825134266028178, 0.076888549405725759, 0.029142433900836595, -0.15039955470091237, 0.031885383748681551, -0.036849541966962523, -0.0090335151587409568, 0.093197750343661123], -0.075400121761051153], [[-0.019456985872773423, 0.34492550602029814, -0.085580857667536245, 1.1710411536252971, -0.29660686639492939, -0.39241176722993321, 0.054434520898320238, 0.80443734093500463, 0.051728582172760912, 0.041088729944448121, -0.057246015153181506, -0.098767569674687195, 0.036465417287937285, 0.11034756929537173, -0.11395338218577326], -0.085219239227286858], [[-0.14231106663132562, 0.44184632243492411, 0.47752304900707943, 0.16236003198198581, -0.18651407493988231, 0.33350107876263191, -0.11721668576444982, 0.39118530497284021, 0.12129555555447177, 0.086273883873666884, 0.17795776035727312, -0.3715642737177734, -0.045789414899917474, 0.07615615287210209, 0.14238442427090875], -0.056763246101817377], [[0.062778659062940265, 0.34929379382974429, 0.53605437288068436, 0.13244501373484097, 0.13203014126186671, 0.49908883811679128, -0.56382750092432421, 0.37578910719263359, -0.079515245962285139, 0.02533911740854948, 0.030263265775452797, 0.16348607963670136, -0.13187494888973528, 0.060474841975876825, -0.10904394959216032], -0.031395599343956082]]

In [41]:
for x in range(1, 13):
    for i, y in enumerate([[0, 7],[7,15],[15,22],[22, 35]]):
        ind = date[(date.dt.year == 2015)&(date.dt.month == x)&(date.dt.day > y[0])&(date.dt.day <= y[1])].index
        final_pred[ind] = [0 if x < 0.5 else 1 for x in (np.array(pred.loc[ind, :]) * coef_list2015[(x-1)*4 + i][0]).sum(axis = 1) + coef_list2015[(x-1)*4 + i][1]]

In [42]:
coef_list2016 = [[[0.083160931053686798, 0.50123045434246438, -0.18436015016358764, 0.68130701919442604, 0.43200682038007177, -0.13604295964040092, 0.86730048043856223, -0.054315340181438576, -0.11990878642468367, -0.050063800461746855, -0.16781948020678739, -0.017739169634362117, -0.053654669444286818, -0.065720715845076993, 0.060483040613759156], -0.1249840501022485], [[0.063187436646098161, 0.30373368200674061, 1.0352977426357544, 0.3126191497028955, -0.077914433527196542, 0.87640946035320866, -0.63982168138475803, -0.14496076399427266, -0.060049558383010515, -0.20683417920784564, -0.0047388938169548589, -0.16135590218796603, -0.15628840179855302, 0.16740171378123109, 0.14767828336246941], -0.027583666045878918], [[-0.030980294648838998, 0.43922046683261201, -0.058868357706043262, 0.18424321650682049, 0.86106773791521762, -0.32550525072686465, 0.11896158762571502, 0.66969483521636475, -0.095674781196959491, 0.032006394265927218, -0.14233265851020718, -0.10707930219848585, 0.031629126969294985, -0.019324452143396537, -0.036687987683005269], -0.079626080992849846], [[-0.13954682033127497, 0.33378976286213791, 0.039806033607806893, 0.66123324614724044, -0.22516298826827527, 0.24567958234692971, 0.10308423516841406, 0.37442856856287487, 0.17653572475170914, 0.14059440547621241, -0.27656781828210741, 0.030288533414691385, 0.058761813190410017, -0.054413963372691443, -0.0020022353426871753], -0.036679859962110484], [[0.037129369093817581, 0.5964270728739044, -0.0081063460338467175, -0.22902343615441942, 0.025090731265902444, 0.99604128550763149, 0.16865515878609616, 0.13849912555419008, -0.16600225119406706, 0.033048310728461927, 0.13042055350669213, -0.0032510487880389738, -0.30229216986185653, -0.022642924492079664, -0.01660969447249272], -0.031204436876102359], [[0.0082779509397315557, 0.27521533089258582, 0.271282282448851, 0.24317783066043919, 0.062882102756682101, 0.4521431499010618, -0.19229296943676083, 0.5437477146870735, 0.058585547353780951, -0.0084156629510135639, -0.055255998117297006, -0.034297359496214271, -0.13039999678299585, 0.037031877430364679, -0.02121886377040233], -0.04764990752098508], [[-0.052013705254242218, 0.51331125256924759, -0.15988689871387099, 0.19022155200745466, -0.092351222909538594, 0.46369523140293517, 0.34804940954857166, 0.43290636769210378, -0.059085881660886547, 0.087312789233614685, -0.10203119330789079, 0.067741651875626863, 0.006542044979097883, -0.079964872107691432, -0.025968044414554387], -0.062214553397731853], [[0.0035154323757134588, 0.38454543946760283, -0.1697376761093464, 0.4045272761907584, 0.73035008984184091, 0.21273459085457591, 0.25397547633361278, -0.03324424351378491, -0.099615443747611124, 0.25953490739354618, -0.12172402518309783, -0.15005876063023216, -0.16200660359045868, -0.040408726661696287, 0.1357592809482786], -0.041462351736194336], [[-0.055766997867973386, 0.4767145176607413, -0.52928267087002179, -0.28042202462113103, 0.54888396859182875, 0.50465770289027989, 0.41277133713636116, 0.53375278268615411, -0.033405733150843719, 0.11803405634009667, 0.063341052706401113, -0.11591676800731615, 0.015009234471315414, -0.023084194758126175, -0.06309272421879808], -0.053198787799453862], [[-0.11921808654820741, 0.55304896032398199, 0.33545462577906782, 0.24652645735943704, 0.044552579593745895, 0.21973740219305932, -0.076825940759151876, 0.40570419920086587, 0.059279502970141673, -0.017626646813853342, -0.10219100754909555, 0.030149974189498718, 0.020593515861067502, 0.06185404788751081, -0.02542982596750254], -0.035417073816853384], [[-0.049542828528010417, 0.49810052204019534, 0.29169250395142649, 0.14836672215851654, -0.46431739526436122, 0.67813958273219033, 0.18972221177940243, 0.3559326649863791, -0.041996124544671193, -0.11896253867737042, 0.1487842103646303, -0.075842527222067302, -0.18728400629354813, 0.075909596907412891, -0.0019456230359987114], -0.018278768600546447], [[-0.10262823373263355, 0.65191373057167312, -0.28743238537415644, 0.3620766146781168, 0.38958765020601949, 0.017297579199162694, 0.20795136345319695, 0.19773175650810898, 0.076873259695250731, 0.16377980648044524, -0.083889894106966401, 0.0058007973684544578, -0.04766633765452899, 0.026744298136452127, -0.030190417104037574], -0.012864659958101266], [[-0.044112450776202093, 0.3664592386316341, 0.59732017641677249, 0.22072828001157588, 0.54930686941293161, -0.21040086570602037, -0.320286820585927, 0.16497839913200574, 0.06654955973086793, 0.0042008847447502218, -0.082302011852698892, -0.072833137017210337, 0.11822863790833205, 0.040761949445283441, -0.0063984967606624255], 0.041235476248580705], [[0.078435481397622769, 0.37244994152640953, 0.58707362776076422, 0.37490143249356256, -0.25283669292671007, 0.60866130292090204, -0.1970030047798168, -0.021329057359658829, 0.018989108798100673, -0.11011685335915292, -0.060082501704998348, -0.037443957435357278, -0.15762130581769557, 0.11426169779263162, 0.07190937929381036], -0.021064875515049541], [[0.0034718108656329578, 0.57915256686658578, 0.29528623742650506, -0.050229880493790402, 0.48137810862273306, 0.25318696244880917, -0.15889571006608449, 0.16483440770352081, -0.059730792487845241, 0.093544024981333074, -0.12247159270970034, 0.075771507207590461, -0.096402151710678813, -0.022497849716211576, 0.036354367030831419], 0.005256366888516828], [[-0.023820576680877423, 0.78225853472656826, 0.011634343713431342, 0.019918416317100285, 0.557084254430666, 0.27156835264849705, 0.14575719682470084, -0.088897624975371042, -0.063732953886952048, -0.039590585757928537, -0.052638053599495027, -0.1870774941584803, -0.040966366635034501, 0.098049128640642746, 0.088011347176140398], -0.021608780483270751], [[-0.061349462068088743, 0.69183746903156507, 0.52172715479450382, 0.48187658768522434, -0.099606104627660508, -0.052401242419038541, 0.024573034101930763, 0.23738518382344564, -0.032962004746863116, -0.12605462236670967, -0.036538133880784029, -0.14126295053073362, 0.14844289442390313, 0.008468265486780327, -0.027186130473260628], -0.033833884151762861], [[-0.11431828195368751, 0.78393055757029084, -0.063003189365440804, 0.18182597873019035, -0.33994031661063867, 0.7460911636953973, 0.32971674305189375, -0.04141630449778666, 0.0062870280295277969, 0.073761911986879036, -0.073507477054227055, 0.085813879520226566, 0.10732331935558642, -0.023845436016895677, -0.11301867089636497], -0.0054055816321251227], [[-0.054753180065529136, 0.72907257976196072, 0.51853520325796476, -0.17125328017481417, 0.35257103677379797, 0.30815676910938594, -0.46331790288173313, 0.079081445127745398, -0.020775639058208617, -0.065812688743678599, -0.015409164790581326, -0.094174344553187739, 0.0008859535771960636, 0.18820843876930005, 0.10425469702505108], 0.056803781410861676], [[-0.042467685625824328, 0.82658628356417985, -0.092932697163839367, 0.022071698051833677, 0.61328725281874952, -0.3824315690142488, 0.3787196213319835, 0.18888077541879378, -0.056751968974047287, -0.11727758155060736, 0.027543187445467854, 0.045457019178591185, -0.047819813632006034, 0.020668298110510047, 0.041497997852390389], 0.024869402799663931], [[-0.064757052637594945, 0.61254843023201899, 0.20086584118803563, 0.21615529630668404, -0.24447282050211117, 0.55194291330325529, -0.009869904195014656, -0.22518268192949908, 0.033520839040034556, -0.075252534891125988, -0.09619332773586535, -0.16578170784589868, 0.12373269435445577, 0.051440396724725945, 0.13820583675161502], 0.07605182599347754], [[-0.095896018600558228, 0.69788715734204487, 0.21436145719045058, 0.83706643929778035, 0.47507517220924905, -0.2908287955029823, 0.18132404698344207, -0.6709551186955669, 0.029008135445491634, -0.020478235431542147, -0.14937658392775088, -0.048159972223858927, 0.047785031619034357, 0.015409964290556966, 0.19223101919119823], 0.046328812007278453], [[-0.14139965440116536, 1.1099453120571932, 0.90215970671662171, -0.22271025596956984, -0.018733690889584986, 0.27713791517741265, -0.015171447692745841, -0.44736962141944825, -0.13945812177327943, -0.16770232269774765, -0.18555121833590268, 0.3340515092066168, -0.019131701690440761, 0.04779454714153375, 0.17032034003113261], -0.0011406461477660446], [[-0.15814004966474193, 0.76510545034432464, 0.55809107865746277, 0.055086482390308111, 1.6874616017811952, -0.98119567509265881, -0.22255726211598903, -0.42610664307362767, 0.048872138661632203, 0.086992973690962261, 0.22112821840946606, -0.31818104317672624, 0.032161073706369003, -0.016595663155686613, 0.17860319823197202], 0.061364613680657654], [[-0.085409697642654284, 0.87040181141592043, 1.1053281722579182, 0.13828450653163327, -1.2316974212995426, -0.0061250907444628522, 0.11930851679229618, -0.29960478093678317, 0.0077587108370841262, -0.29725813009402691, 0.25100538067247935, 0.15072089763164931, 0.14909301157086774, -0.018689890304061096, 0.39481458468156305], 0.05877411487918438], [[-0.29984373974635936, 0.71934734076942719, 1.2392092995547836, 0.41184431977829539, 0.13052097731042822, 0.012981282398934396, -0.77084668605305651, -0.23923303071710911, 0.24613616288030205, 0.0071792330569403484, -0.16906316486569095, 0.18095148458834975, -0.12071531335216135, 0.18887601960560274, 0.051885666612439096], 0.069759828760088682], [[-0.15449120581946524, 0.64951964246975047, 0.16969084588187366, 0.70820451021752118, 0.68696831944577208, -0.47619883509347344, -0.29090557003820378, 0.07620809485410926, 0.19548669409941749, -0.059913958975032836, -0.11821757647728848, -0.19305873749916524, -0.13176045531817471, 0.15929252439240482, 0.056679345616207244], 0.11912041223397452], [[0.006180404871175979, 0.80464282037114809, -0.33033917021567527, -0.14848881979217166, -1.235242142750355, 1.3095685396707577, 0.6957242895856508, -0.02989187842375074, -0.10674055798263112, -0.037619586416044015, 0.011956177131142276, 0.050958002969719307, -0.0086747554764927681, -0.11553937525330471, 0.32522194317886177], 0.029034802249806424], [[-0.1657612836095966, 0.94073016547909261, 0.51058101741405593, 0.030484186580697514, -0.47335063751875994, 0.57147407316464671, 0.37793039521913463, -0.47295122425549962, 0.010816584966879941, 0.097829543387180029, -0.21309623583345366, 0.074870851146988271, -0.058132337538235268, -0.24332889978101008, 0.42768616534437087], 0.038796184224211938], [[0.025647555447089326, 0.72666569404040504, 1.6463192615309852, 0.15034166562043572, -0.067317069313437283, 0.46532622741236068, -0.86103506347136061, -0.75741516656446195, -0.035803335172834738, -0.22717670542117901, 0.071836839974666111, -0.084342850750207754, -0.054927106726417874, 0.20650080761349554, 0.20007689716806881], 0.093074451237209366], [[-0.001959021207822406, 0.83239308122272682, 0.37957376389069414, 0.18619450498571441, 0.56580540139778224, -0.33440574530327416, -0.22863859727686675, -0.29546311597557634, -0.075875965058126549, -0.13685162690406641, 0.08330235469297663, -0.15623644870280184, 0.03650658244675975, 0.16184825625987359, 0.33483923528636689], 0.051801286141569425], [[-0.15561910248092847, 0.9655331160147993, 0.19024631202086678, -0.16389065866071933, 0.49217663484548924, 0.59772856889259007, -0.66386712615445909, 0.085032757491798894, -0.062744636941233445, 0.067025521253281539, 0.069696799220337891, 0.047764491738702577, -0.20580601665107331, 0.090321642139469582, 0.020765245239603963], 0.017690729166233488], [[-0.096453655024564017, 0.58878378008258592, 0.91393894691688504, 0.34938317289662602, 0.039152078643877972, 0.039398004355004235, -1.0468286783459875, 0.46362537171815743, 0.02312687381619816, 0.086659296247407092, -0.093363837264509641, 0.17609181669056617, -0.064349293969083843, 0.19115432937995736, -0.060071639048318637], 0.092949401936266773], [[0.018536111824581453, 0.70288133456920732, 0.1572960624779724, 0.94921016378166667, 0.65848222506481491, -0.53704710893855245, -0.29893180864523983, -0.40740458137528102, -0.14933227609016425, -0.011001519678496552, -0.031467342092052764, 0.016334637904423882, -0.011023627493020494, 0.046868387035175152, 0.077048979071970614], 0.096965255896032387], [[-0.25259118431472216, 0.8233415869510412, 0.4199431033875719, 1.1768474473476098, -1.2356704052210177, 0.14276615677208457, -0.071037668353966799, 0.41782871470689475, 0.15248698131333682, -0.24720613484926324, -0.42648857055296946, 0.36562680947974546, -0.085172934683006352, 0.29547776416200666, -0.06802360170871613], 0.052760631572301764], [[-0.072576660742084367, 0.81651417087679246, -0.3211034844306131, 0.30928733494137484, -0.09012779788680797, -0.50078664788211746, 0.9512547206547276, -0.0032189580080161928, -0.10700584083014469, 0.032073855929995732, -0.010406086197703862, 0.069831232786959546, 0.34587216310976254, 0.093766240427648195, -0.17325996366570007], 0.096218432324126135], [[-0.28432041279606995, 0.63988723059029973, 1.0836922161217861, 0.98661564214995512, -0.12433327393925117, -0.46263556657516319, -0.065707352399842003, -0.31079427858663244, 0.23194998789726029, -0.33864363672233078, 0.31520027787319682, -0.21545976552161242, -0.040905762230344422, -0.00748873833591579, -0.21221297709348941], 0.088021580093375407], [[-0.16089707161042341, 0.8965822415267819, 0.4103863205396423, 0.59468823344320354, 0.04254317994671461, 0.28553571860940874, 0.21152992243231228, -0.68788481353454334, 0.064970151509591206, -0.22785316880554574, -0.051201941851013566, -0.32032223026961659, 0.15010684940229679, -0.063439537422950276, 0.22006490290322889], 0.025322682321992973], [[0.11182075799149772, 0.65411818318708925, 1.3129187937616515, 0.85288827805273559, -1.6246409006550993, 0.74593025778158539, -0.38871261440224841, -0.087892290411351701, -0.045656229767157573, 0.045928349904733434, -0.16611837605758883, 0.2315789223043605, -0.34715191537467827, 0.054190747068566192, 0.18518850753887883], 0.05267236980336254], [[-0.13344781089068675, 0.63515225114238016, 0.77325119481004267, -0.23308661481801735, 0.2008554964948821, -0.095572234935413447, -0.76559358529034227, 1.1408187820540743, 0.011940503111879601, -0.012057379165902993, 0.066549237185619392, 0.13054067356138055, -0.15333355924898628, 0.024779068153564121, -0.28546668205502557], 0.09847076037014274], [[-0.21037641281310179, 0.9288051581200526, -0.66322254241331191, 0.24471045837055735, -0.19194499568561604, 1.1597561102737344, -0.003107297478991633, 0.26577554130079833, -0.012955549233710062, 0.57645607258796061, -0.27205638140095012, 0.098248048134944366, -0.41718998126023216, -0.080263742634118618, -0.20794407164281067], 0.033673328415926296], [[-0.14994541548372922, 0.98141226935245029, -0.27950378846317891, -0.32498007101383392, -0.35353773357469331, 0.5436907607315189, 0.67536406628105428, 0.11623154500823252, -0.00067651238963972737, 0.13859895364541563, 0.34966577728333448, 0.13896685765843331, -0.39184623610185354, -0.17269902859927255, 0.00040508572409858612], 0.028379878546068105], [[-0.14590691431003053, 1.0464832814251701, 1.0481929470578977, -0.48657408078753095, 0.62350112672748947, 0.93308813773062682, -0.34721584768203095, -1.4288201702593981, -0.095042162810413611, -0.069649525643022514, 0.18041281864798286, 0.0086063826300428747, -0.21864748321611804, 0.12624215537856975, 0.43201176983660594], -0.020934269528173011], [[0.07207680342415454, 0.77697734766041227, 0.3067820573261868, 0.022557231517119136, 0.82240771305419913, -0.19652115180943242, 0.07599237305462464, -0.36763840644908552, -0.013013836366206788, -0.26191350689882448, -0.45585536331995213, -0.11287556111335956, 0.38283161849607267, 0.025787016595057471, 0.036475336578099038], -0.005342340330847406], [[-0.017361177160982864, 1.042984596048806, -0.22705026056619793, -0.10520380866992814, 0.37986986873844969, 0.0035414727513486122, 0.93228250277674085, -0.56238540287508332, -0.23254838672858535, 0.11408832920138845, 0.41925891479783384, -0.055394342682305586, -0.41360226254919463, 0.082108661979596098, 0.21278205018642582], -0.0055409687694392695], [[-0.023808340221479832, 0.84635535835645104, 0.27922958125037833, 0.936967869657783, -0.40313567111024923, 0.84330653662802091, -0.42275541949917705, -0.24602082764493866, -0.076037549032303448, 0.18946227321254394, -0.63949378370955645, 0.0076838455058896304, 0.2848136733716235, -0.32201195158638829, 0.05865693087106294], -0.0040226517553044738], [[-0.084129629439401846, 0.38795226024982232, 1.1803963363130143, 1.9883352651032504, -1.9821675141870865, 0.22611224235732835, -0.69729948680873344, -0.075019732689127927, 0.23349343981311343, 0.26662333349037559, -0.48588083112512215, 0.41253348601745166, 0.22478142391797751, -0.24390460187141016, -0.18026241639775309], 0.081950757990531559], [[-0.047550062368741451, 0.88350187140437808, 0.56199307124951803, -0.096428489165694131, 0.90391694368564324, -1.4389011253249979, 0.39303569862021248, 0.69111563392488917, -0.025263595517992787, -0.049169933732915005, 0.16752516730843969, -0.30151511955072863, 0.23789111228216764, -0.48602221325105943, 0.23101391574902952], 0.00063119675804385045]]

In [43]:
for x in range(1, 13):
    for i, y in enumerate([[0, 7],[7,15],[15,22],[22, 35]]):
        ind = date[(date.dt.year == 2016)&(date.dt.month == x)&(date.dt.day > y[0])&(date.dt.day <= y[1])].index
        final_pred[ind] = [0 if x < 0.5 else 1 for x in (np.array(pred.loc[ind, :]) * coef_list2016[(x-1)*4 + i][0]).sum(axis = 1) + coef_list2016[(x-1)*4 + i][1]]

In [44]:
coef_list2017 = [[[-0.092260279685306562, 0.47993713095700025, 1.3627279414391347, 0.19273462880948466, -0.43111286718320307, 0.3093180618980409, -0.63330613216205289, -0.06358364220948258, 0.19092647149070691, -0.22550467829007492, -0.0096726092674919162, 0.032662864635206401, -0.31034612227919123, 0.36000601525052756, 0.072208252512741211], 0.1033118679714411], [[0.026020136865209237, 0.71385675075166954, 0.19154883685548907, 0.34416777217129813, -0.95574217469666189, 0.14653896310740583, 0.54406576756292635, 0.50815570748139871, -0.11041845651374511, 0.15500194489430491, -0.30283088541950576, -0.11729819887503543, 0.11573965958550625, -0.037944605033069845, 0.23614815740095207], 0.032610559808463446], [[0.058485304818391169, 1.0464212109432975, 0.19961840811742557, 0.30101063937092576, 0.90901659787261513, 0.10254210837610717, 0.50387515072918676, -1.3332534333399786, -0.29452778524554846, -0.037687102120534721, -0.1671236775444731, -0.059676630015028098, -0.16696044545586608, 0.078660354501412511, 0.30556879269033232], 0.022689027579702148], [[-0.20008981539808449, 0.68803052136755405, 0.7171076414175982, -0.089068886422192503, 0.45442419258068401, 0.65089805716290039, -1.289926063724629, 0.31339860971881217, 0.16465953479286172, -0.060096924040990818, 0.26641700319021283, -0.10774731392237336, -0.26224116097107819, 0.1765668954172569, 0.013030401338264719], 0.044476192980860224]]

In [45]:
for i, y in enumerate([[0, 7],[7,15],[15,22],[22, 35]]):
        ind = date[(date.dt.year == 2017)&(date.dt.month == 1)&(date.dt.day > y[0])&(date.dt.day <= y[1])].index
        final_pred[ind] = [0 if x < 0.5 else 1 for x in (np.array(pred.loc[ind, :]) * coef_list2017[i][0]).sum(axis = 1) + coef_list2017[i][1]]

In [47]:
ans = pd.DataFrame()
ans['project_id'] = test.project_id
ans['final_status'] = final_pred.astype('int8')
#ans.to_csv('predict_final_roman.csv', index = None)

In [49]:
offline = pd.read_csv('../../../offline_testcase.csv')

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
accuracy_score(offline.final_status, ans.final_status)

0.76434255101236903